In [1]:
import numpy as np
import cv2
import glob

try:
    from PIL import Image
except ImportError:
    import Image

In [2]:
# For each pixel create a vector containing its Saturation, Value and x- and y-coordinates
def pixelVectors(hsv):
    row_idx = 0
    column_idx = 0
    # Create an empty matrix for storing pixel data
    pixel_data = np.zeros([len(hsv), len(hsv[0]), len(hsv[0][0])+1])
    for row in hsv:
        for pixel in row:
            # Put the vector for that pixel in the matrix
            pixel_data[row_idx][column_idx] = [pixel[1],pixel[2],column_idx,row_idx]
            column_idx+=1
        column_idx = 0
        row_idx+=1
    # Flatten the matrix with the pixel data into a list of vectors
    pixel_data = pixel_data.reshape(len(pixel_data)*len(pixel_data[0]), len(pixel_data[0][0]))
    return pixel_data

In [3]:
# Normalise the vectors using means and standard deviations
def normalisePixels(pixel_data):
    means = np.mean(pixel_data, axis=0)
    stds = np.std(pixel_data, axis=0)
    normed_pixels = []
    for row in pixel_data:
        normed_pixels.append((row-means)/stds)
    return normed_pixels,means,stds

In [6]:
# The cluster with the logo is the one with higher Saturation and Value - this chooses colourful regions over black and white
def findLogoCluster(cluster_centers)
    if (cluster_centers[0][0]**2+cluster_centers[0][1]**2) > (cluster_centers[1][0]**2+cluster_centers[1][1]**2):
        logo_cluster = 0
    else:
        logo_cluster = 1
    return logo_cluster

array([[ -8.51038903e-02,   3.31276740e-02,   4.33720122e-03,
          1.04248021e-02],
       [  1.01547814e+01,  -3.95286613e+00,  -5.17524287e-01,
         -1.24391007e+00]])

In [ ]:
# With red points mark on the original image what points are in the cluster identified as the logo
red = [0, 0, 255]
def showLogoCluster(normed_pixels, kmeans, means, stds, img)
    for point in normed_pixels:
        # Convert to the original points using the means and standard deviations
        if kmeans.predict([point]) == logo_cluster:
            if point[2]*stds[2]+means[2] > len(img)-2:
                xcoord = len(img)-1
            else:
                xcoord = point[2]*stds[2]+means[2]
            if point[3]*stds[3]+means[3] > len(img[0])-2:
                ycoord = len(img[0])-1
            else:
                ycoord = point[3]*stds[3]+means[3]
            img[int(round(ycoord)), int(round(xcoord))] = red
    cv2.imshow('img',img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()    

In [8]:
# Read all images into a list
image_list = []
for ext in ["jpg","gif","png"]:
    for filename in glob.glob('C:/users/szyma/Documents/Data Science projects/Logos/logo_images/*.%s' % ext) : #assuming png
        img = cv2.imread(filename)
        if type(img)!=None:
            hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
            image_list.append(hsv_img)
#img = cv2.imread('100082743.png')
#hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

error: OpenCV(4.0.0) C:\projects\opencv-python\opencv\modules\imgproc\src\color.cpp:181: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [5]:
# Run K-means clustering on the normalised vectors
from sklearn.cluster import KMeans
# Assume 2 clusters - one for logo and one for the rest of the document
kmeans = KMeans(n_clusters=2, random_state=0).fit(normed_pixels)
cluster_centers = kmeans.cluster_centers_